In [1]:
import os
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops'

In [3]:
import pandas as pd
data = pd.read_csv('artifacts/data_ingestion/data.csv')
data.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Creditability                      1000 non-null   int64
 1   Account Balance                    1000 non-null   int64
 2   Duration of Credit (month)         1000 non-null   int64
 3   Payment Status of Previous Credit  1000 non-null   int64
 4   Purpose                            1000 non-null   int64
 5   Credit Amount                      1000 non-null   int64
 6   Value Savings/Stocks               1000 non-null   int64
 7   Length of current employment       1000 non-null   int64
 8   Instalment per cent                1000 non-null   int64
 9   Sex & Marital Status               1000 non-null   int64
 10  Guarantors                         1000 non-null   int64
 11  Duration in Current address        1000 non-null   int64
 12  Most valuable availab

In [6]:
data.isnull().sum()

Creditability                        0
Account Balance                      0
Duration of Credit (month)           0
Payment Status of Previous Credit    0
Purpose                              0
Credit Amount                        0
Value Savings/Stocks                 0
Length of current employment         0
Instalment per cent                  0
Sex & Marital Status                 0
Guarantors                           0
Duration in Current address          0
Most valuable available asset        0
Age (years)                          0
Concurrent Credits                   0
Type of apartment                    0
No of Credits at this Bank           0
Occupation                           0
No of dependents                     0
Telephone                            0
Foreign Worker                       0
dtype: int64

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:PATH
    local_data_file:Path
    all_schema:dict

In [13]:
from src.datascience.constants import *
from src.datascience.utils.common import create_directories, read_yaml

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAM_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])    

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            local_data_file = config.local_data_file,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )            

        return data_validation_config


In [16]:
import os
from src.datascience import logger

In [23]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)->bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.local_data_file)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()
            report_lines = []

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    report_lines.append(f"Column:{col}, {validation_status}")
                else:
                    validation_status=True
                    report_lines.append(f"Column:{col}, {validation_status}")

            # Write all results at once
            with open(self.config.STATUS_FILE, 'w') as f:
                for line in report_lines:
                    f.write(line + '\n')

            return validation_status

        except Exception as e:
            raise e

In [24]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
     raise e

[2025-09-16 23:18:08,176: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-16 23:18:08,177: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-16 23:18:08,186: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-09-16 23:18:08,189: INFO: common]: created directory at: artifacts
[2025-09-16 23:18:08,190: INFO: common]: created directory at: artifacts/data_validation
